In [1]:
import pandas as pd
import numpy as np
import os
import sys
import re

from matplotlib import pyplot as plt
from dotenv import load_dotenv

sys.path.append("../")

from scripts import search

load_dotenv()
plt.style.use('Solarize_Light2')

# Setting default DPI, pulling it from dotenv if it exists, setting it on 100 if not

try:
    pc_dpi = int(os.getenv('DPI'))
except TypeError:
    pc_dpi = 100
if pc_dpi is None:
    pc_dpi = 100


/var/folders/3s/s8sp6jwn6qs02jfxbgjc7c_40000gn/T/ipykernel_54751/759101277.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df_meteorites = pd.read_pickle(filepath_or_buffer="../datasets/metbull_data.pkl")


In [3]:
display(df_meteorites.head(n=20))


,name,year,country,type,mass,URL,numeric_id
0,Denader 001,2022,Mali,H4-melt breccia,5330.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,1
1,Hassi Khebi 001,2022,Algeria,C3-ung,500.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,1
2,Qaen 001,2016,Iran,L6,21000.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,1
3,Aachen,1880,Germany,L5,21.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
4,Aammiq,2000,Lebanon,H6,596.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
5,Aarhus,1951,Denmark,H6,720.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
6,Aba,1927,Japan,Pseudometeorite,0.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
7,Aba Panu,2018,Nigeria,L3,160000.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
8,Abadla,2015,Algeria,Eucrite-cm,222.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
9,Abadla 002,2022,Algeria,CM2,800.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,2


# <u> Use cases :</u>

## <u> Name :</u>
### <u>case 1 : exact match : lets say I want Catalina 20</u>

In [5]:
result = search.search_by_name(df=df_meteorites, name_query="Catalina", numeric_range=20)
display(result)


,name,year,country,type,mass,URL,numeric_id
7820,Catalina 020,2010,Chile,L6,2080.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,20


### case 2 : lets say I want Yamato 100 -> 150 :

In [6]:
# case insensitive, so I can put caps wherever I want it doesnt matter

result = search.search_by_name(df=df_meteorites, name_query="YaMaTO", numeric_range=(100, 150))
display(result.head(n=5))
display(result.tail(n=5))


,name,year,country,type,mass,URL,numeric_id
77577,Yamato 000100,2000,Antarctica,<NA>,0.20,https://www.lpi.usra.edu/meteor/metbull.php?co...,100
77578,Yamato 000101,2000,Antarctica,<NA>,0.74,https://www.lpi.usra.edu/meteor/metbull.php?co...,101
77579,Yamato 000102,2000,Antarctica,H5-6,12.50,https://www.lpi.usra.edu/meteor/metbull.php?co...,102
77580,Yamato 000103,2000,Antarctica,<NA>,3.31,https://www.lpi.usra.edu/meteor/metbull.php?co...,103
77581,Yamato 000104,2000,Antarctica,<NA>,1.49,https://www.lpi.usra.edu/meteor/metbull.php?co...,104


,name,year,country,type,mass,URL,numeric_id
77623,Yamato 000146,2000,Antarctica,<NA>,1.06,https://www.lpi.usra.edu/meteor/metbull.php?co...,146
77624,Yamato 000147,2000,Antarctica,<NA>,2.83,https://www.lpi.usra.edu/meteor/metbull.php?co...,147
77625,Yamato 000148,2000,Antarctica,<NA>,1.38,https://www.lpi.usra.edu/meteor/metbull.php?co...,148
77626,Yamato 000149,2000,Antarctica,H4,84.60,https://www.lpi.usra.edu/meteor/metbull.php?co...,149
77627,Yamato 000150,2000,Antarctica,L5,142.20,https://www.lpi.usra.edu/meteor/metbull.php?co...,150


## <u>Types :</u>

In [7]:
# Pretty straighforward, case insensitive, as scrapped on metbull :

result = search.search_by_type(df=df_meteorites, query="l3")
print(result.__len__())
display(result.head(n=5))


834


,name,year,country,type,mass,URL,numeric_id
7,Aba Panu,2018,Nigeria,L3,160000.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
409,Acfer 341,2002,Algeria,L3,132.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,341
415,Acfer 347,2001,Algeria,L3,1165.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,347
635,Al Huqf 068,2009,Oman,L3,165.4,https://www.lpi.usra.edu/meteor/metbull.php?co...,68
723,Alatage Mountain 049,2023,China,L3,568.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,49


## <u>Country :</u>

In [8]:
# pretty much the same :

result = search.search_by_country(df=df_meteorites, query="France")
print(result.__len__())
display(result.head(n=5))


90


,name,year,country,type,mass,URL,numeric_id
62,Abbans-Dessous,2015,France,LL6,3.12,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
520,Agen,1814,France,H5,30000.00,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
562,Aire-sur-la-Lys,1769,France,<NA>,6800.00,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
670,Alais,1806,France,CI1,6000.00,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
735,Alby sur Chéran,2002,France,Eucrite-mmict,252.00,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>


### Pretty basic but it gets the job done.

# <u>Now let's try to join a few meteorites in a single dataframe to request further details via MetBull :</u>

In [9]:
group_1 = search.search_by_name(df=df_meteorites, name_query="catalina", numeric_range=(30, 60))

# We can also stack the searches, so here, lets take yamato 1000-1100, type h4 :
group_2 = search.search_by_type(
    df=search.search_by_name(df=df_meteorites, name_query="yamato", numeric_range=(1000, 1100)),
    query="h4"
    )

group_3 = search.search_by_country(df=df_meteorites, query="lebanon")


In [10]:
# Concat the result :
df_search = pd.concat([group_1, group_2, group_3]).drop_duplicates().reset_index(drop=True)

display(df_search)


,name,year,country,type,mass,URL,numeric_id
0,Catalina 030,2010,Chile,H5,214.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,30
1,Catalina 031,2010,Chile,L6,1178.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,31
2,Catalina 032,2010,Chile,H4,1107.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,32
3,Catalina 033,2010,Chile,L6,211.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,33
4,Catalina 034,2010,Chile,LL5,20.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,34
...,...,...,...,...,...,...,...
58,Yamato 001091,2000,Antarctica,H4,6.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,1091
59,Yamato 001095,2000,Antarctica,H4,7.5,https://www.lpi.usra.edu/meteor/metbull.php?co...,1095
60,Aammiq,2000,Lebanon,H6,596.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
61,Aphaca,<NA>,Lebanon,Doubtful meteorite,0.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
